In [4]:
import pandas as pd

customers = pd.read_csv('data/customers.csv', parse_dates=['signup_date'])
products = pd.read_csv('data/products.csv')
orders = pd.read_csv('data/orders.csv', parse_dates=['order_date'])

In [10]:
# 결측치 확인
print("=== orders 결측치 확인 ===")
print( orders.isna().sum() )
print("=== products 결측치 확인===")
print( products.isna().sum() )
print("=== customers 결측치 확인 ===")
print( customers.isna().sum() )

=== orders 결측치 확인 ===
order_id      0
user_id       0
order_date    0
product_id    0
qty           0
unit_price    1
region        2
dtype: int64
=== products 결측치 확인===
product_id    0
category      1
brand         2
dtype: int64
=== customers 결측치 확인 ===
user_id        0
signup_date    0
gender         2
age_group      1
region         2
dtype: int64


In [14]:
dup_orders = orders.duplicated().sum()
print(f"중복등록된 주문 수 : {dup_orders}")

중복등록된 주문 수 : 0


In [17]:
print("=== 총 정리 ===")
print(f"customer 수 : {len(customers)}")
print(f"orders 수 : {len(orders)}")
print(f"products 수 : {len(products)}")

print(f"중복등록된 주문 수 : {dup_orders}")
print(f"orders의 결측치 수 : {orders.isna().sum().sum()}")

=== 총 정리 ===
customer 수 : 150
orders 수 : 2500
products 수 : 80
중복등록된 주문 수 : 0
orders의 결측치 수 : 3


In [ ]:
# 결측치 처리
print("=== 고객 데이터 결측치 ===")
missing_cus = customers.isna().sum()

for col in missing_cus.index:
  ratio = missing_cus[col] / len(customers) * 100
  print(f"{col} : {ratio:.1f}%")

=== 고객 데이터 결측치 ===
user_id : 0.0%
signup_date : 0.0%
gender : 1.3%
age_group : 0.7%
region : 1.3%


In [27]:
# 결측치가 들어있는 행들을 보고 싶음
print( orders[ orders.isna().any(axis=1) ] )
print( products[ products.isna().any(axis=1) ] )
print( customers[ customers.isna().any(axis=1) ] )

    order_id user_id order_date product_id  qty  unit_price region
13   o000014    u061 2024-05-10      p0002    3         NaN  Seoul
53   o000054    u034 2024-02-21      p0078    5     12000.0    NaN
103  o000104    u124 2024-05-20      p0017    1     12000.0    NaN
   product_id category brand
13      p0014     tool   NaN
33      p0034      NaN  Duro
73      p0074     food   NaN
    user_id signup_date gender age_group   region
10     u011  2023-07-12    NaN       40s  Incheon
11     u012  2023-12-23      F       NaN    Seoul
12     u013  2023-09-26      M       10s      NaN
48     u049  2023-09-05    NaN       10s  Gwangju
143    u144  2023-05-30      M       30s      NaN


In [41]:
# 고객의 성별 결측치 처리
gender_mode = customers['gender'].mode()[0]
customers['gender'] = customers['gender'].fillna(gender_mode)

In [43]:
# 고객의 나이대 결측치 처리, 지역도 처리
age_group_mode = customers['age_group'].mode()[0]
customers['age_group'] = customers['age_group'].fillna(age_group_mode)

region_mode = customers['region'].mode()[0]
customers['region'] = customers['region'].fillna(region_mode)

In [47]:
print(f"고객 데이터의 성별의 결측치는 {gender_mode}로 나이대는 {age_group_mode} 지역은 {region_mode} 대체")
print(f"결측치 처리 후 결측치 개수 : {customers.isna().sum().sum()}")

고객 데이터의 성별의 결측치는 M로 나이대는 10s 지역은 Gwangju 대체
결측치 처리 후 결측치 개수 : 0


In [51]:
# 상품 데이터의 결측치 처리
# 카테고리와 브랜드 둘다 최빈값으로 대체
cate_mode = products['category'].mode()[0]
products['category'] = products['category'].fillna(cate_mode)

brand_mode = products['brand'].mode()[0]
products['brand'] = products['brand'].fillna(brand_mode)

In [61]:
# 주문 데이터 결측치 처리
mean_price = orders['unit_price'].mean().round(0)
orders['unit_price'] = orders['unit_price'].fillna(mean_price)

orders_region_mode = orders['region'].mode()[0]
orders['region'] = orders['region'].fillna(orders_region_mode)

In [ ]:
# 결측치 처리가 완료된 후 처리 결과
# 처리전 결측치 개수 -> 처리 후 개수
# 각각 어떤 값으로 대체해준건지

In [64]:
# 매출액이 필요하므로 수량*단가를 계산한 파생변수를 추가
# 월별 통계를 내야하므로 월데이터만 가지고있는 파생변수를 추가
orders = orders.assign(
  total = orders['qty'] * orders['unit_price'],
  order_month = orders['order_date'].dt.to_period('M').astype(str)
)

orders.head()

,order_id,user_id,order_date,product_id,qty,unit_price,region,total,order_month
0,o000001,u066,2024-02-02,p0010,4,9000.0,Incheon,36000.0,2024-02
1,o000002,u072,2024-04-10,p0013,4,9000.0,Incheon,36000.0,2024-04
2,o000003,u112,2024-06-21,p0016,4,20000.0,Incheon,80000.0,2024-06
3,o000004,u130,2024-06-19,p0008,4,20000.0,Seoul,80000.0,2024-06
4,o000005,u079,2024-03-06,p0026,1,30000.0,Incheon,30000.0,2024-03


In [66]:
print(f"최소 주문금액 : {orders['total'].min()}")
print(f"최대 주문금액 : {orders['total'].max()}")
print(f"평균 주문금액 : {orders['total'].mean()}")
print(f"총 매출액 : {orders['total'].sum()}")
print(f"기간 : {orders['order_month'].min()} ~ {orders['order_month'].max()}")

최소 주문금액 : 5000.0
최대 주문금액 : 150000.0
평균 주문금액 : 45077.1736
총 매출액 : 112692934.0
기간 : 2024-01 ~ 2024-06


In [78]:
# 데이터 병합 : 주문 + 고객 + 상품 데이터
df = orders.merge(
  customers[['user_id', 'region']], on='user_id', suffixes=('', '_cust')
)

df = df.merge(products, on='product_id')

df.head()

,order_id,user_id,order_date,product_id,qty,unit_price,region,total,order_month,region_cust,category,brand
0,o000001,u066,2024-02-02,p0010,4,9000.0,Incheon,36000.0,2024-02,Seoul,fashion,Duro
1,o000002,u072,2024-04-10,p0013,4,9000.0,Incheon,36000.0,2024-04,Seoul,digital,Eon
2,o000003,u112,2024-06-21,p0016,4,20000.0,Incheon,80000.0,2024-06,Seoul,digital,Bando
3,o000004,u130,2024-06-19,p0008,4,20000.0,Seoul,80000.0,2024-06,Seoul,beauty,Ciel
4,o000005,u079,2024-03-06,p0026,1,30000.0,Incheon,30000.0,2024-03,Gwangju,tool,Ciel


In [79]:
print("== 병합 후 데이터 확인 ==")
print(f"행 개수 : {len(df)}")
print(f"열 개수 : {len(df.columns)}")
print(f"목록 : {df.columns}")
print(f"결측치 수 : {df.isna().sum().sum()}")

== 병합 후 데이터 확인 ==
행 개수 : 2500
열 개수 : 12
목록 : Index(['order_id', 'user_id', 'order_date', 'product_id', 'qty', 'unit_price',
       'region', 'total', 'order_month', 'region_cust', 'category', 'brand'],
      dtype='object')
결측치 수 : 0


In [83]:
# 월별 매출
monthly = df.groupby('order_month', as_index=False)['total'].sum()

print(f"=== 월별 매출 분석 ===")
print(f"매출 분석 기간 : {monthly['order_month'].min()} ~ {monthly['order_month'].max()}")
print(f"월 평균 매출 : {monthly['total'].mean()}")
print(f"최고 매출 월 : {monthly.loc[ monthly['total'].idxmax(), 'order_month']} / {monthly['total'].max()}")
print(f"최저 매출 월 : {monthly.loc[ monthly['total'].idxmin(), 'order_month']} / {monthly['total'].min()}")

=== 월별 매출 분석 ===
매출 분석 기간 : 2024-01 ~ 2024-06
월 평균 매출 : 18782155.666666668
최고 매출 월 : 2024-03 / 19372000.0
최저 매출 월 : 2024-05 / 17893934.0


In [90]:
monthly['total_grow'] = monthly['total'].pct_change() * 100
monthly

,order_month,total,total_grow
0,2024-01,18823000.0,NaN
1,2024-02,18782000.0,-0.217819
2,2024-03,19372000.0,3.141306
3,2024-04,18668000.0,-3.634111
4,2024-05,17893934.0,-4.146486
5,2024-06,19154000.0,7.041861


In [91]:
# 지역, 카테고리에 해당하는 피벗 테이블
# 지역별 카테고리별 매출액의 계산결과를 하나의 표로 만들어줌
px = pd.pivot_table(
  df, index='region', columns='category', values='total', aggfunc='sum'
)
px

category,beauty,digital,fashion,food,tool
region,,,,,
Busan,5179000.0,6418000.0,4485000.0,3231000.0,3897000.0
Daegu,5439000.0,5958000.0,3176000.0,3073000.0,4113000.0
Gwangju,4904000.0,5321000.0,3809000.0,3221000.0,4680000.0
Incheon,5759000.0,6932000.0,4064000.0,2602000.0,4356000.0
Seoul,4913000.0,5211934.0,3506000.0,3065000.0,5380000.0


In [100]:
# 지역별 매출 순위 출력
region_total = px.sum(axis=1).sort_values(ascending=False)
print("=== 지역별 매출액 순위 ===")
for i, (region, total) in enumerate(region_total.items()):
  print(f"{i+1}. {region} : {total}")

=== 지역별 매출액 순위 ===
1. Incheon : 23713000.0
2. Busan : 23210000.0
3. Seoul : 22075934.0
4. Gwangju : 21935000.0
5. Daegu : 21759000.0


In [101]:
# 카테고리 매출 순위 출력
category_total = px.sum().sort_values(ascending=False)
print("=== 카테고리 매출액 순위 ===")
for i, (category, total) in enumerate(category_total.items()):
  print(f"{i+1}. {category} : {total}")

=== 카테고리 매출액 순위 ===
1. digital : 29840934.0
2. beauty : 26194000.0
3. tool : 22426000.0
4. fashion : 19040000.0
5. food : 15192000.0


In [105]:
max_region, max_category = px.stack().idxmax()
max_total = px.loc[max_region, max_category]

print("=== 최고 매출액 ===")
print(f"지역 : {max_region}, 카테고리 : {max_category}, 매출액 : {max_total}")


=== 최고 매출액 ===
지역 : Incheon, 카테고리 : digital, 매출액 : 6932000.0


In [110]:
# 매출액이 높은 상품들
top = df.groupby( ['product_id', 'brand', 'category'], as_index=False )['total'].sum()\
        .sort_values('total', ascending=False).head(10)
top

,product_id,brand,category,total
5,p0006,Eon,beauty,2687000.0
45,p0046,Aster,tool,2101000.0
38,p0039,Bando,digital,2025000.0
48,p0049,Ciel,fashion,1917000.0
64,p0065,Duro,beauty,1912000.0
51,p0052,Ciel,digital,1878000.0
43,p0044,Aster,beauty,1837000.0
23,p0024,Aster,beauty,1829000.0
21,p0022,Duro,beauty,1817000.0
35,p0036,Duro,digital,1812000.0


In [114]:
print("== 상위 10개 상품 == ")
print(f"top-10의 총 매출 :  {top['total'].sum():,.0f}원")
print(f"전체 매출 대비 top-10 비중 : {top['total'].sum() / df['total'].sum() * 100:.2f}%")

== 상위 10개 상품 == 
top-10의 총 매출 :  19,815,000원
전체 매출 대비 top-10 비중 : 17.58%


In [117]:
print("== top-10 브랜드 분포 ==")
top_brand = top['brand'].value_counts()
for brand, count in top_brand.items():
  print(f"{brand} : {count}개")

== top-10 브랜드 분포 ==
Aster : 3개
Duro : 3개
Ciel : 2개
Eon : 1개
Bando : 1개


In [118]:
print("== top-10 카테고리 분포 ==")
top_category = top['category'].value_counts()
for category, count in top_category.items():
  print(f"{category} : {count}개")

== top-10 카테고리 분포 ==
beauty : 5개
digital : 3개
tool : 1개
fashion : 1개


In [121]:
print(f"1등 상품 매출액 : {top.iloc[0]['total']}")
print(f"10등 상품 매출액 : {top.iloc[-1]['total']}")

1등 상품 매출액 : 2687000.0
10등 상품 매출액 : 1812000.0
